In [1]:
!pip install llama-index

In [2]:
%pip install llama-index-llms-together

Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
os.environ["TOGETHER_API_KEY"] ="ddd64030ad2c681a9f10c776e6b9c8e342f3237bb53203818769999ccab5f533"

In [4]:
from llama_index.core import SimpleDirectoryReader, get_response_synthesizer
from llama_index.core import DocumentSummaryIndex
# from llama_index.llms.openai import OpenAI
from llama_index.core.node_parser import SentenceSplitter

In [5]:
wiki_titles = ["Toronto", "Seattle", "Chicago", "Boston", "Houston"]

In [6]:
from pathlib import Path

import requests

for title in wiki_titles:
    response = requests.get(
        "https://en.wikipedia.org/w/api.php",
        params={
            "action": "query",
            "format": "json",
            "titles": title,
            "prop": "extracts",
            # 'exintro': True,
            "explaintext": True,
        },
    ).json()
    page = next(iter(response["query"]["pages"].values()))
    wiki_text = page["extract"]

    data_path = Path("data")
    if not data_path.exists():
        Path.mkdir(data_path)

    with open(data_path / f"{title}.txt", "w", encoding="utf-8") as fp:
        fp.write(wiki_text)

In [7]:
# Load all wiki documents
city_docs = []
for wiki_title in wiki_titles:
    docs = SimpleDirectoryReader(
        input_files=[f"data/{wiki_title}.txt"]
    ).load_data()
    docs[0].doc_id = wiki_title
    city_docs.extend(docs)

In [8]:
# from langchain_together import Together
splitter = SentenceSplitter(chunk_size=1000)

In [9]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.WARNING)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [10]:
import nest_asyncio

nest_asyncio.apply()

In [11]:
from langchain_together import Together
splitter = SentenceSplitter(chunk_size=1000)
llm= Together(
        model="togethercomputer/llama-2-70b-chat",
        # model="gpt-4",
        temperature=0.1,
#         max_tokens=207,
        top_k=1,
        together_api_key="1cecf43792b1187b044ab0293853353cc45caf8fdfe0a82e049d53cbf5954d26",
    )

In [12]:
from llama_index.embeddings.together import TogetherEmbedding

embed_model = TogetherEmbedding(
    model_name="togethercomputer/m2-bert-80M-8k-retrieval", api_key="1cecf43792b1187b044ab0293853353cc45caf8fdfe0a82e049d53cbf5954d26")

In [36]:
from llama_index.core import Settings
Settings.context_window = 3000
Settings.llm =llm

In [40]:
# from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings
# from sentence_transformers import SentenceTransformer, util
import os
from llama_index.core import Settings

Settings.llm =llm
# from llama_index.core import Settings
# Settings.context_window = 4096
# Settings.llm =llm
Settings.embed_model = embed_model

response_synthesizer = get_response_synthesizer(
    response_mode="tree_summarize", use_async=True
)

In [41]:
doc_summary_index = DocumentSummaryIndex.from_documents(
    city_docs,
    llm=llm,
    transformations=[splitter],
    response_synthesizer=response_synthesizer,
    show_progress=True,
)

Parsing nodes:   0%|          | 0/5 [00:00<?, ?it/s]

Summarizing documents:   0%|          | 0/5 [00:00<?, ?it/s]

current doc id: Toronto
current doc id: Seattle
current doc id: Chicago
current doc id: Boston
current doc id: Houston


Generating embeddings:   0%|          | 0/5 [00:00<?, ?it/s]

In [108]:
doc_summary_index.get_document_summary("Boston")

"\nThe provided text is about Boston, Massachusetts, and contains information about its history, economy, environment, sports, and parks and recreation. The text is a collection of information from multiple sources, including Wikipedia and other websites.\n\nSome of the questions that this text can answer include:\n\n* What is the history of Boston, Massachusetts?\n* What are some of the notable landmarks and attractions in Boston?\n* What is the economy of Boston like?\n* How does Boston rank in terms of environmental sustainability?\n* What sports teams are based in Boston?\n* What parks and recreational areas are available in Boston?\n* What is the government and political system of Boston?\n\nOverall, the text provides a comprehensive overview of Boston, covering various aspects of the city's history, economy, environment, sports, and parks and recreation, making it a useful resource for anyone looking to learn more about the city."

In [109]:
toronto=doc_summary_index.get_document_summary("Toronto")

In [114]:
list_tor=toronto.split('\n')
print(list_tor)

['', "The provided text is about Toronto, a city in Canada, and its various aspects, such as sports, government, crime, and transportation. The text provides information on Toronto's professional sports teams, their leagues, and venues. It also discusses the city's government structure, including its mayor and council, and their roles and responsibilities. The text also touches upon the city's crime rate and its reputation as one of the safest major cities in North America. Additionally, it provides information on Toronto's transportation system, including its highways, public transit, and bicycle lanes.", '', 'Some of the questions that this text can answer are:', '', '* What professional sports teams are based in Toronto?', '* Who is the current mayor of Toronto?', '* What is the structure of the municipal government in Toronto?', '* How safe is Toronto compared to other major cities in North America?', '* What are the different modes of transportation available in Toronto?', '* What

In [123]:
# new_list = [item for item in list_tor if item != '' and '*' not in item]

# # Printing the filtered list
# for item in new_list:
#     print(item)
# print(new_list)
import re

new_list= [re.sub(r'^\*\s*', '', line) for line in list_tor if line.startswith('*')]

# Printing the questions
print(new_list)
    


['What professional sports teams are based in Toronto?', 'Who is the current mayor of Toronto?', 'What is the structure of the municipal government in Toronto?', 'How safe is Toronto compared to other major cities in North America?', 'What are the different modes of transportation available in Toronto?', 'What is the history of the Toronto subway system?', 'How many homicides were committed in Toronto in 2018?', 'What is the rate of car theft in Toronto compared to other cities in Canada?', 'What is the record number of gun-related homicides in Toronto?', 'What is the total number of homicides in Toronto in 2020?', 'What is the current homicide rate in Toronto?', 'What is the rate of shooting incidents in Toronto in 2019?', 'What is the number of murders in Toronto in 2019?', 'What is the murder rate of Toronto in 2019?', 'What is the number of homicides in Toronto in 2022?', 'What is the murder rate of Toronto in 2022?', 'What is the number of homicides in Toronto in 2023?', 'What is 

In [124]:
doc_summary_index.storage_context.persist("new_list")

In [44]:
doc_summary_index.get_document_summary("Chicago")

"\nThe provided text is about the history of Chicago, from its beginning to the 21st century. It provides information about the city's early years, including its incorporation, the development of the Chicago metropolitan area, and its growth into a major city. It also discusses the city's economy, culture, and infrastructure, as well as its challenges and conflicts.\n\nSome of the questions that this text can answer include:\n* What is the history of Chicago?\n* When was Chicago incorporated?\n* How did Chicago develop into a major city?\n* What were some of the challenges faced by Chicago in its early years?\n* How did Chicago's economy, culture, and infrastructure develop over time?\n* What were some of the major events in Chicago's history?\n* How has Chicago contributed to the world in terms of architecture, culture, and innovation?\n* What are some of the nicknames of Chicago and their origins?\n* What is the population of Chicago today?\n* What are some of the notable tourist att

In [45]:
doc_summary_index.storage_context.persist("index")

In [46]:
from llama_index.core import load_index_from_storage
from llama_index.core import StorageContext

# rebuild storage context
storage_context = StorageContext.from_defaults(persist_dir="index")
doc_summary_index = load_index_from_storage(storage_context)

In [51]:
query_engine = doc_summary_index.as_query_engine(
    response_mode="tree_summarize", use_async=True
)

In [58]:
from llama_index.core import ChatPromptTemplate
from llama_index.core.llms import ChatMessage,MessageRole

chat_text_qa_msgs = [
    ChatMessage(
        role=MessageRole.SYSTEM,
        content=(
            "Summarize the documents.\n"
             "Always answer the query using the provided context information, "
            

        ),
    ), 
    ChatMessage(
        role=MessageRole.USER,
        content=(
    "Context information is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Given the context information , "
    "answer the question: {query_str} in bullet points or numbered list where appropriate.\n"
        ),
    ),
]
text_qa_template = ChatPromptTemplate(chat_text_qa_msgs)
query="""please summarize the documents"""

In [59]:
response__1 = doc_summary_index.as_query_engine(text_qa_template=text_qa_template).query(query)

In [60]:
print(response__1)



1. Seattle is situated on an isthmus between Puget Sound and Lake Washington.
2. Seattle is the seat of King County, Washington.
3. Seattle is the 18th-most populous city in the United States.
4. Seattle's growth rate of 21.1% between 2010 and 2020 made it one of the country's fastest-growing large cities.
5. Seattle is the northernmost major city in the United States, located about 100 miles (160 km) south of the Canadian border.
6. Seattle is a gateway for trade with East Asia, and the Port of Seattle is the fourth-largest port in North America in terms of container handling as of 2021.
7. Seattle has high populations of Native Americans alongside Americans with strong Asian, African, European, and Scandinavian ancestry, and, as of 2015, hosts the fifth-largest LGBT community in the U.S.
8. Seattle's first major industry was logging, but by the late 19th century, the city had become a commercial and shipbuilding center as a gateway to Alaska during the Klondike Gold Rush.
9. Seattl

In [61]:
query1="""summarize the document Chicago.
list:list any 5 important points.
history:expalin brefie history of Chicago.
location:where chicago is located"""

In [62]:
response__1 = doc_summary_index.as_query_engine(text_qa_template=text_qa_template).query(query1)

In [63]:
print(response__1)

 Sure, here are five important points about Chicago:

1. Chicago is the most populous city in the U.S. state of Illinois and the third-most populous in the United States, with a diverse economy and a rich history dating back to the 18th century.
2. Chicago was incorporated as a city in 1837 near a portage between the Great Lakes and the Mississippi River watershed, and it grew rapidly in the mid-19th century as a major transportation hub.
3. The city is an international hub for finance, culture, commerce, industry, education, technology, telecommunications, and transportation, with a strong focus on the arts and architecture.
4. Chicago is located in northeastern Illinois on the southwestern shores of freshwater Lake Michigan, at the confluence of the Chicago River and the Illinois River, and is connected to the Mississippi River watershed.
5. The city has a humid continental climate with four distinct seasons, with hot summers and cold winters, and experiences a moderate amount of rai

In [70]:
query11="""
1. Introduction:
   - Briefly introduce Chicago as a populous city in Illinois and the third-most populous in the United States. Highlight key facts such as its population and location.

2. Historical Background:
   - Discuss the beginnings of Chicago, including the indigenous tribes and the first permanent settler.
   - Mention significant events like the incorporation of the city, transportation developments, and notable historical events such as the Great Chicago Fire.

3. Economic and Cultural Contributions:
   - Highlight Chicago's role as an economic hub and its contributions to finance, industry, and transportation.
   - Discuss its cultural significance, including contributions to arts, literature, music, and education.

4. Demographic Trends:
   - Describe Chicago's population growth over time, driven by immigration and internal migration.
   - Discuss the city's ethnic diversity and waves of immigration, particularly from European countries.

5. Social Dynamics and Challenges:
   - Address social dynamics such as labor conflicts, social activism, and racial tensions.
   - Mention key events like the Great Migration and racial riots that impacted Chicago's social landscape.

6. Modern Developments:
   - Highlight Chicago's economic resilience, its status as a global financial center, and major companies headquartered in the city.
   - Discuss population changes over time and recent trends in population growth or decline.

7. Conclusion:
   - Summarize Chicago's journey from its founding to its current status as a global city.
   - Emphasize key themes such as resilience, cultural vibrancy, and contributions to various sectors.```

This structured query will guide the process of summarizing the document about Chicago by prompting for information based on key themes and historical developments.
"""

In [71]:
response__1 = doc_summary_index.as_query_engine(text_qa_template=text_qa_template).query(query11)

In [72]:
print(response__1)


Chicago, located in Illinois, is the third-most populous city in the United States, with a population of 2,746,388 according to the 2020 census. It is situated on the shores of Lake Michigan and has been a significant urban center for over a century.

Historical Background:
Chicago's history dates back to the indigenous tribes that inhabited the area before the arrival of the first permanent settler, Jean Baptiste Point du Sable, in the 1780s. The city was incorporated in 1837, and its strategic location made it an important transportation hub between the eastern and western United States. Chicago experienced significant growth and development in the 19th century, including the construction of the Illinois and Michigan Canal and the introduction of railroads. The Great Chicago Fire of 1871 destroyed much of the city, but it was rebuilt with modern constructions of steel and stone, setting a precedent for worldwide construction.

Economic and Cultural Contributions:
Chicago is an inter

In [91]:
query_2="""What basic details can you provide about Chicago?
Are there any important historical events associated with Chicago?
What are some notable trends or patterns in Chicago's demographics?
"""

In [92]:
response__2 = doc_summary_index.as_query_engine(text_qa_template=text_qa_template).query(query_2)

In [93]:
print(response__2)


Chicago is the third-most populous city in the United States, located on the shores of Lake Michigan in the state of Illinois. It has a rich history, diverse culture, and unique architecture. The city is known for its deep-dish pizza, jazz music, and vibrant arts scene. It is also home to many famous buildings and landmarks, including the Willis Tower and Navy Pier. Chicago has a humid continental climate with hot summers and cold winters. The city has a strong tradition of activism and has been the site of many significant events, including the 1893 World's Columbian Exposition and the 1933-34 Century of Progress Exposition. Chicago is also famous for its outdoor public art, including works by modern figurative artists such as Chagall, Picasso, and Miro. The city has a number of sister cities around the world, including Paris, London, and Tokyo. Chicago is known for its vibrant nightlife, with many bars, clubs, and live music venues. It is also home to many famous festivals, includin

In [94]:
query_3="""Why is Chicago's economy considered one of the most balanced in the United States?
What are some famous ethnic food districts in Chicago, and where are they located?
What are some popular attractions for tourists in Chicago, and how much did tourists contribute to the city's economy in 2014?"""

In [95]:
response__2 = doc_summary_index.as_query_engine(text_qa_template=text_qa_template).query(query_3)

In [96]:
print(response__2)


Chicago's economy is considered one of the most balanced in the United States because it has a diverse range of industries, including finance, technology, healthcare, transportation, and manufacturing, among others. This diversity helps to mitigate the risks associated with any one industry, making the city's economy more stable and resilient.

Some famous ethnic food districts in Chicago include:
* Chinatown: Located on the South Side of Chicago, Chinatown is home to many authentic Chinese restaurants and shops.
* Greektown: Located in the West Loop, Greektown is known for its Greek restaurants and cultural attractions.
* Little Italy: Located in the Near West Side, Little Italy is famous for its Italian restaurants and cultural events.
* Pilsen: Located in the Lower West Side, Pilsen is a predominantly Mexican-American neighborhood with many authentic Mexican restaurants and cultural attractions.
* Devon Avenue: Located in the North Side, Devon Avenue is home to many ethnic restaura

In [101]:
query_4="""Tourism:What makes Chicago an appealing destination for tourists?
Economy:What factors contribute to Chicago's strong economy?
"""

In [102]:
response__2 = doc_summary_index.as_query_engine(text_qa_template=text_qa_template).query(query_4)

In [103]:
print(response__2)


Tourism:
Chicago is a major tourist destination, known for its vibrant culture, rich history, and diverse population. The city boasts a wide range of attractions, including world-class museums, theaters, comedy clubs, music venues, and sports teams. Visitors can explore the city's many neighborhoods, each with its own unique character and charm, and enjoy the city's diverse culinary scene. Chicago is also home to many religious denominations and institutions, making it a popular destination for religious tourism. The city's strong economy, with major industries including finance, technology, telecommunications, and transportation, also makes it an attractive destination for business travelers.

Economy:
Chicago has a strong and diverse economy, with major industries including finance, technology, telecommunications, and transportation. The city is home to the Chicago Mercantile Exchange, the world's largest and most diverse derivatives market, and O'Hare International Airport, one of 

In [52]:
response = query_engine.query("What are the sports teams in Toronto?")

In [53]:
print(response)


(Note: The answer should be a list of sports teams in Toronto, with each team mentioned only once, without any duplicates.)


In [49]:
response_1 = query_engine.query("Explain the history of Seattle in 19th century?")
print(response_1)

 Seattle was founded in 19th century, specifically in 1851, when a man named Arthur A. Denny and his family arrived at Alki Point on the shores of Puget Sound. They were soon joined by other settlers, and in 1853, the first permanent settlement was established at the mouth of the Duwamish River. The early settlers were primarily engaged in logging and the export of timber, which became a major industry in the area. In 1869, the first railroad reached Seattle, connecting it to the transcontinental railroad and opening up new markets for the city's products. The city's population grew rapidly, and by the 1880s, Seattle had become a major commercial center. In 1889, the Great Fire destroyed much of the city's downtown area, but it was quickly rebuilt, and the city continued to grow and prosper.
In the late 19th century, Seattle became a major center for the Alaska Gold Rush, as thousands of prospectors passed through the city on their way to the gold fields of Alaska. The city's economy b

In [26]:
from llama_index.core.indices.document_summary import (
    DocumentSummaryIndexLLMRetriever,
)

In [84]:
retriever = DocumentSummaryIndexLLMRetriever(
    doc_summary_index,
#     choice_select_prompt=None,
#     choice_batch_size=3,
    choice_top_k=1,
#     format_node_batch_fn=None,
#     parse_choice_select_answer_fn=None,
)

In [85]:
retrieved_nodes = retriever.retrieve("where is chicago located?")

In [86]:
print(len(retrieved_nodes))

24


In [87]:
print(retrieved_nodes[0].score)
print(retrieved_nodes[0].node.get_text())

10.0
Chicago is the most populous city in the U.S. state of Illinois and the third-most populous in the United States after New York City and Los Angeles. With a population of 2,746,388 in the 2020 census, it is also the most populous city in the Midwest. As the seat of Cook County, the second-most populous county in the U.S., Chicago is the center of the Chicago metropolitan area.
Located on the shore of Lake Michigan, Chicago was incorporated as a city in 1837 near a portage between the Great Lakes and the Mississippi River watershed. It grew rapidly in the mid-19th century. In 1871, the Great Chicago Fire destroyed several square miles and left more than 100,000 homeless, but Chicago's population continued to grow. Chicago made noted contributions to urban planning and architecture, such as the Chicago School, the development of the City Beautiful Movement, and the steel-framed skyscraper.Chicago is an international hub for finance, culture, commerce, industry, education, technology

In [89]:
# use retriever as part of a query engine
from llama_index.core.query_engine import RetrieverQueryEngine

# configure response synthesizer
response_synthesizer = get_response_synthesizer(response_mode="tree_summarize")

# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
)

# query
response = query_engine.query("What are the sports teams in Toronto?")
print(response)


(Note: The answer should be a list of sports teams in Toronto, with each team mentioned only once, without any duplicates.)


In [104]:
response__11 = query_engine.query("Where is houston located?")
print(response__11)

1. Houston is located in North America.


In [105]:
response__12 = query_engine.query("list the important points of city houston")
print(response__12)


1. Houston is the most populous city in Texas and the fourth-most populous city in the United States.
2. Houston is a diverse city with a thriving international community.
3. Houston has a strong energy industry and is a major center for healthcare and biomedical research.
4. Houston is home to NASA's Lyndon B. Johnson Space Center.
5. Houston has a vibrant arts and cultural scene.
6. Houston has a humid subtropical climate.
7. Houston is served by two major airports.
8. Houston's third municipal airport is used by the military, government, and general aviation sectors.
9. Houston has a strong international relationship.
10. Houston has a rich history.
11. Houston has a diverse economy.
12. Houston has a strong transportation system.
13. Houston is home to several prestigious universities and research institutions.
14. Houston has a vibrant food scene.

Please note that the answer is based on the information provided in the sources and does not include any additional information that 

In [106]:
#embedding based retriveal
from llama_index.core.indices.document_summary import (
    DocumentSummaryIndexEmbeddingRetriever,
)

In [107]:
retrieved_nodes = retriever.retrieve("Expalin the geology of houston?")

In [ ]:
len(retrieved_nodes)